# Chapter 5: Incidence Matrices

Incidence matrices S (source) and D (destination) factor the adjacency matrix: S × D^T = A.

In [ ]:
import graphblas as gb
from graphblas import Matrix, semiring
import networkx as nx
import matplotlib.pyplot as plt

## Graph as Edge List

In [ ]:
# Edges: (source, destination)
edges = [(0,1), (0,2), (1,2), (2,3)]
n_nodes = 4
n_edges = len(edges)

print("Edges:")
for i, (s, d) in enumerate(edges):
    print(f"  e{i}: {s} -> {d}")

In [ ]:
G = nx.DiGraph(edges)
pos = {0: (0,1), 1: (1,1), 2: (0.5,0), 3: (1.5,0)}
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=500, font_size=16, arrows=True)
plt.show()

## Incidence Matrices

In [ ]:
# S[node, edge] = 1 if node is source of edge
# D[node, edge] = 1 if node is destination of edge

S_rows, D_rows = [], []
for edge_idx, (src, dst) in enumerate(edges):
    S_rows.append((src, edge_idx))
    D_rows.append((dst, edge_idx))

S = Matrix.from_coo(
    [r[0] for r in S_rows], [r[1] for r in S_rows], 
    [1]*n_edges, nrows=n_nodes, ncols=n_edges, dtype=int
)
D = Matrix.from_coo(
    [r[0] for r in D_rows], [r[1] for r in D_rows],
    [1]*n_edges, nrows=n_nodes, ncols=n_edges, dtype=int
)

print("Source incidence matrix S (nodes x edges):")
print(S)
print("\nDestination incidence matrix D (nodes x edges):")
print(D)

## S × D^T = A

In [ ]:
# Multiply S × D^T to get adjacency matrix
A = S.mxm(D.T, semiring.plus_times).new()
print("Adjacency matrix A = S × D^T:")
print(A)

In [ ]:
# Verify against direct construction
A_direct = Matrix.from_coo(
    [e[0] for e in edges], [e[1] for e in edges],
    [1]*n_edges, nrows=n_nodes, ncols=n_nodes, dtype=int
)
print("Direct adjacency matrix:")
print(A_direct)
print(f"\nMatrices equal: {A.isequal(A_direct)}")

## Multigraphs

Incidence matrices naturally handle parallel edges (multiple edges between same nodes).

In [ ]:
# Multigraph: two edges from 0 to 1
multi_edges = [(0,1), (0,1), (1,2)]
n_multi_edges = len(multi_edges)

S_multi = Matrix.from_coo(
    [e[0] for e in multi_edges], list(range(n_multi_edges)),
    [1]*n_multi_edges, nrows=3, ncols=n_multi_edges, dtype=int
)
D_multi = Matrix.from_coo(
    [e[1] for e in multi_edges], list(range(n_multi_edges)),
    [1]*n_multi_edges, nrows=3, ncols=n_multi_edges, dtype=int
)

A_multi = S_multi.mxm(D_multi.T, semiring.plus_times).new()
print("Multigraph adjacency (with edge counts):")
print(A_multi)
print("\nA[0,1] = 2: two parallel edges from 0 to 1")